In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import funcs.utils as utils
import funcs.plotting as plot
import funcs.amyloid as amyloid
import scipy
from tableone import TableOne


In [36]:
PROCESSED_DIR = "data/processed"

### 1. Load Data
---

In [42]:
# Raw Data
data_df = pd.read_csv(os.path.join(PROCESSED_DIR,"AL_with_ccp_03.tsv"), sep="\t", index_col=0).rename(columns=amyloid.ddict_unclean)
data_df = data_df.dropna(subset="cluster")

# Fix Dates
data_df = pd.concat([pd.to_datetime(data_df[amyloid.dates][var], format="mixed") for var in amyloid.dates], axis=1, keys=amyloid.dates).join(
    data_df.drop(amyloid.dates, axis=1)  
)

# Not imputed
X = pd.read_csv(os.path.join(PROCESSED_DIR, "AL_for_ccp_02.tsv"), sep='\t', index_col=0).rename(columns=amyloid.ddict_unclean)

# Imputed
Xi_median = pd.read_csv("data/imputed/median_qvars_01.tsv", sep="\t", index_col=0).rename(columns=amyloid.ddict_unclean)
Xi_knn = pd.read_csv("data/imputed/knn_qvars_01.tsv", sep="\t", index_col=0).rename(columns=amyloid.ddict_unclean)
Xi_mice = pd.read_csv("data/imputed/mice_qvars_05.tsv", sep="\t").rename(columns={'X24_hr_UTP':'24_hr_UTP'}).rename(columns=amyloid.ddict_unclean)

In [43]:
# Order cluster subtypes
from pandas.api.types import CategoricalDtype

data_df["cluster"] = data_df["cluster"].astype(CategoricalDtype(categories=["Low","Intermediate","High"], ordered=True))
data_df["cluster4"] = data_df["cluster4"].astype(CategoricalDtype(categories=["Low","Low-Intermediate","Intermediate","High"], ordered=True))

# Sex
data_df["Sex"] = data_df["Sex"].apply(lambda x: "F" if x=="female" else "M")

# Collapse Race
data_df["Race"] = data_df["Race"].apply(lambda x: "Other" if x in ['American_Indian_Alaska_Native','Multiracial','Native_Hawaiian_Pacific', 'Unknown/other'] else x)

for x in amyloid.amyloid_ros + amyloid.amyloid_symptoms:
    data_df[x] = data_df[x].apply(lambda x: True if x in ["involved","yes"] else False)

In [44]:
rename = {
    "Bone marrow plasma cells (%)":"BMPC (%)", 
    "Kappa or lambda PCD":"LC Isotype",
    "BU (BNP-based) cardiac staging":"BU Stage (2019)",
    "time":"OS (yr)"}

data_df["Renal Stage (Palladini)"] = data_df["Renal Stage (Palladini)"].apply(lambda x: {"Stage I":"I", "Stage II":"II", "Stage III":"III", np.nan:None, None:None}[x])
data_df["BU (BNP-based) cardiac staging"] = data_df["BU (BNP-based) cardiac staging"].apply(lambda x: {"stage I":"I", "stage II":"II", "stage III":"III", "stage IIIb":"IIIb", np.nan:None, None:None}[x])


In [50]:
utils.get_median_os(data_df, duration="time"), utils.get_median_os(data_df, groupby="cluster", duration="time"), utils.get_median_os(data_df, groupby="Era", duration="time")

(3.989048596851472,
 {'High': 1.566050650239562,
  'Intermediate': 4.027378507871321,
  'Low': 7.112936344969199},
 {'Era_1-2': 2.97056810403833,
  'Era_3': 3.3894592744695413,
  'Era_4': 5.938398357289528})

### 2. Abstract Table
---

In [13]:
# Table for abstract
columns = ["OS (yr)","LC Isotype","dFLC","BNP","Troponin","eGFR","24-hr UTP","Albumin","BU Stage (2019)"]
categorical = ["LC Isotype","BU Stage (2019)",] 
groupby = ["cluster"]
nonnormal = ["OS (yr)","dFLC","BNP", "Troponin","eGFR","24-hr UTP"]

mytable = TableOne(data_df, columns, categorical, groupby, nonnormal, pval=True, rename=rename)
mytable.to_html("onetable.html")
print(mytable.tabulate(tablefmt="github"))

/Users/sanand/anaconda3/envs/r4/lib/python3.10/site-packages/tableone/tableone.py:370: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  self._groupbylvls = sorted(data.groupby(groupby).groups.keys())  # type: ignore


|                           |      | Missing   | Overall               | Low                  | Intermediate            | High                 | P-Value   |
|---------------------------|------|-----------|-----------------------|----------------------|-------------------------|----------------------|-----------|
| n                         |      |           | 2074                  | 689                  | 822                     | 563                  |           |
| OS (yr), median [Q1,Q3]   |      | 0         | 2.4 [0.7,6.4]         | 3.7 [1.2,7.8]        | 2.6 [0.7,6.8]           | 1.2 [0.4,3.4]        | <0.001    |
| LC Isotype, n (%)         | K    | 0         | 450 (21.7)            | 200 (29.0)           | 140 (17.0)              | 110 (19.5)           | <0.001    |
|                           | L    |           | 1624 (78.3)           | 489 (71.0)           | 682 (83.0)              | 453 (80.5)           |           |
| dFLC, median [Q1,Q3]      |      | 517       | 86.6 [26.

/Users/sanand/anaconda3/envs/r4/lib/python3.10/site-packages/tableone/tableone.py:927: FutureWarning: The provided callable <function mean at 0x104e37c70> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df_cont = pd.pivot_table(cont_data,
/Users/sanand/anaconda3/envs/r4/lib/python3.10/site-packages/tableone/tableone.py:927: FutureWarning: The provided callable <function median at 0x104f73490> is currently using DataFrameGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  df_cont = pd.pivot_table(cont_data,
/Users/sanand/anaconda3/envs/r4/lib/python3.10/site-packages/tableone/tableone.py:927: FutureWarning: The provided callable <built-in function min> is currently using DataFrameGroupBy.min. In a future version of pandas, the provided callable will be used dir

### 3. Table 1
---

In [58]:
columns = ["Age","Sex","Race","Kappa or lambda PCD", "Primary organ","cluster"] + list(Xi_mice.columns) + amyloid.amyloid_symptoms

categorical = ["Sex","Race","Kappa or lambda PCD", "Primary organ"] + amyloid.amyloid_symptoms
groupby = ["cluster"]
nonnormal = []

mytable = TableOne(data_df, columns, categorical, groupby, nonnormal, 
                   pval=True, overall=True, 
                   decimals = {'WBC':2, 'Hemoglobin':2, 'Troponin': 3, 'Calcium':2, 
                               'Bone marrow plasma cells (%)':2, 'Uric acid':2, 'Albumin':2, 'kappa:lambda ratio':2},
                   rename=amyloid.tableone_names)

print(mytable.tabulate(tablefmt="github"))
mytable.to_excel('tables/Table1.xlsx')

/Users/sanand/anaconda3/envs/r4/lib/python3.10/site-packages/tableone/tableone.py:370: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  self._groupbylvls = sorted(data.groupby(groupby).groups.keys())  # type: ignore
/Users/sanand/anaconda3/envs/r4/lib/python3.10/site-packages/tableone/tableone.py:927: FutureWarning: The provided callable <function mean at 0x104e37c70> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df_cont = pd.pivot_table(cont_data,
/Users/sanand/anaconda3/envs/r4/lib/python3.10/site-packages/tableone/tableone.py:927: FutureWarning: The provided callable <function median at 0x104f73490> is currently using DataFrameGroupBy.median. In a future version of pan

|                                  |             | Missing   | Overall         | Low             | Intermediate    | High            | P-Value   |
|----------------------------------|-------------|-----------|-----------------|-----------------|-----------------|-----------------|-----------|
| n                                |             |           | 2074            | 689             | 822             | 563             |           |
| Age, mean (SD)                   |             | 0         | 60.9 (10.4)     | 61.0 (10.0)     | 60.7 (11.0)     | 61.0 (10.2)     | 0.824     |
| Sex, n (%)                       | F           | 0         | 810 (39.1)      | 276 (40.1)      | 342 (41.6)      | 192 (34.1)      | 0.015     |
|                                  | M           |           | 1264 (60.9)     | 413 (59.9)      | 480 (58.4)      | 371 (65.9)      |           |
| Race, n (%)                      | Asian       | 0         | 47 (2.3)        | 20 (2.9)        | 18 (2.2)        | 9

['Mayo  2004 cardiac biomarker staging', 'BU (BNP-based) cardiac staging']

In [72]:
columns = ["BU (BNP-based) cardiac staging","Renal Stage (Palladini)","cluster"]

categorical = ["BU (BNP-based) cardiac staging","Renal Stage (Palladini)"]
groupby = ["cluster"]
nonnormal = []

mytable = TableOne(data_df, columns, categorical, groupby, nonnormal, 
                   pval=True, overall=True,
                   rename={
                       "BU (BNP-based) cardiac staging":"Cardiac Stage (BU 2019)",
                       "Renal Stage (Palladini)":"Renal Stage (Palladini)"
                       })

print(mytable.tabulate(tablefmt="github"))
mytable.to_excel('tables/Table1B.xlsx')

/Users/sanand/anaconda3/envs/r4/lib/python3.10/site-packages/tableone/tableone.py:370: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  self._groupbylvls = sorted(data.groupby(groupby).groups.keys())  # type: ignore


/Users/sanand/anaconda3/envs/r4/lib/python3.10/site-packages/tableone/tableone.py:1449: UserWarning: Order variable not found: cluster4
  warnings.warn("Order variable not found: {}".format(k))


|                                |      | Missing   | Overall    | Low        | Intermediate   | High       | P-Value   |
|--------------------------------|------|-----------|------------|------------|----------------|------------|-----------|
| n                              |      |           | 2074       | 689        | 822            | 563        |           |
| Cardiac Stage (BU 2019), n (%) | I    | 993       | 305 (28.2) | 172 (43.5) | 124 (34.1)     | 9 (2.8)    | <0.001    |
|                                | II   |           | 459 (42.5) | 187 (47.3) | 159 (43.7)     | 113 (35.1) |           |
|                                | III  |           | 151 (14.0) | 29 (7.3)   | 56 (15.4)      | 66 (20.5)  |           |
|                                | IIIb |           | 166 (15.4) | 7 (1.8)    | 25 (6.9)       | 134 (41.6) |           |
| Renal Stage (Palladini), n (%) | I    | 265       | 792 (43.8) | 344 (59.6) | 153 (20.1)     | 295 (62.8) | <0.001    |
|                       